In [633]:
import numpy as np
from scipy.sparse import coo_matrix
import cvxpy as cp
import math
from gurobi_optimods.min_cost_flow import min_cost_flow_scipy

In [634]:
def prepare_dataset(n, skew_function):
    res = []
    fa = []
    ya = []
    for _ in range(n):
        f = np.random.uniform()
        y = int(np.random.uniform() > 1 - skew_function(f))
        fa.append(f)
        ya.append(y)
    return (np.array(fa), np.array(ya))

In [635]:
S_calibrated = []
for i in range(10):
    S_calibrated.append(prepare_dataset(2**10, lambda x:(x+0.01)))

In [637]:
class node(object):
    def __init__(self, l, r):
        self.l = l
        self.r = r
        self.set = None
        self.add = None
        self.left = None
        self.right = None

def compose(root, c, func = True):
    if func:
        root.set = c
        root.add = None
    else:
        if root.add != None:
            root.add += c
        elif root.set != None:
            root.set += c
        else:
            root.add = c

class segment_tree(object):
    def __init__(self, size):
        def createTree(l, r):
            if l > r:
                return None
            if l == r:
                n = node(l, r)
                return n
            mid = (l + r) // 2       
            root = node(l, r)
            root.left = createTree(l, mid)
            root.right = createTree(mid+1, r)
            return root
        self.root = createTree(0, size-1)
        self.size = size
            
    def apply(self, l, r, c, func = True):                    
        def applyHelper(root, l, r, c, func = True):
            if root.l > r or root.r <l:
                return
            if root.l >= l and root.r <= r:
                compose(root, c, func)
                return
            if root.add != None:
                compose(root.left, root.add, False)
                compose(root.right, root.add, False)
            elif root.set != None:
                compose(root.left, root.set, True)
                compose(root.right, root.set, True)                
            applyHelper(root.left, l, r, c, func)
            applyHelper(root.right, l, r, c, func)
            root.add = None
            root.set = None
            return 
        
        return applyHelper(self.root, l, r, c, func)
    
    
    def access(self, ind):
        def accessHelper(root, i):
            if root.l == root.r:
                if root.add != None:
                    return root.add
                elif root.set != None:
                    return root.set
                else:
                    return 0
            if root.add != None:
                compose(root.left, root.add, False)
                compose(root.right, root.add, False)
                root.add = None
            elif root.set != None:
                return root.set
            if i <= root.left.r:
                return accessHelper(root.left, i)
            else:
                return accessHelper(root.right, i)
        return accessHelper(self.root, ind)
    
    def add(self, l, r, c):
        self.apply(l, r, c, False)
        return
    
    def set(self, l, r, c):
        self.apply(l, r, c, True)
        return    
    
    def binary_search(self, c):
        low = 0
        high = self.size - 1
        mid = 0
        t = 0
        while low <= high:
            mid = (high + low) // 2
            if self.access(mid) < c:
                low = mid + 1

            elif self.access(mid) > c:
                high = mid - 1
            else:
                return mid-1
        return high


In [638]:
def DP(S):
    (x_list, y_list) = S
    n = len(x_list)
    indices = np.argsort(x_list)
    x_list = x_list[indices]
    y_list = y_list[indices] 
    threshold = np.zeros((n-2,2))
    demands = -np.array((y_list - x_list)/n)
    cost = (x_list- np.roll(x_list,1,axis=0))[1:len(x_list)]

    point = np.zeros(n+1)
    if -demands[0] <= 0:
        point[0] = -demands[0]
        point[1] = 0
    else:
        point[0] = 0
        point[1] = -demands[0]

    # find vertices
    shift = np.zeros(n-2)
    for i in range(1,n-2):
        shift[i] = shift[i-1] -demands[i]
        point[i+1] = -shift[i]

    point[0:n-1] = point[0:n-1] + shift[n-3]-demands[n-2]
    point[n-1] = demands[n-1]
    ind = np.argsort(np.argsort(point))
    point = np.sort(point)
    
    # find slopes
    l = int(ind[0])
    r = int(ind[1])
    slopes = segment_tree(n+2)
    if -demands[0] <= 0:
        slopes.set(0, r, 1-cost[0])
        slopes.add(0, l, -2)
        slopes.set(r+1, n+1, 1+cost[0])
        threshold[0][0] = -demands[0]
        threshold[0][1] = 0
    else:
        slopes.set(0, r, -1+cost[0])
        slopes.add(0, l, -2*cost[0])
        slopes.set(r+1, n+1, 1+cost[0])
        threshold[0][0] = 0
        threshold[0][1] = -demands[0]

    for i in range(1, n-2):
        slopes.set(0,l, -1)
        slopes.set(r+1, n+1, 1)
        slopes.add(0, int(ind[i+1]), -cost[i])
        slopes.add(int(ind[i+1]+1), n+1, cost[i])
        l = slopes.binary_search(-1)
        r = slopes.binary_search(1)
        threshold[i][0] = point[l] + shift[i] - shift[n-3] +demands[n-2]
        threshold[i][1] = point[r] + shift[i] - shift[n-3] +demands[n-2]
    slopes.set(0,l, -1)
    slopes.set(r+1, n+1, 1)
    slopes.add(0, int(ind[n-1]), -1)
    slopes.add(int(ind[n-1]+1), n+1, 1)
    slopes.add(0, int(ind[n]), -cost[n-2])
    slopes.add(int(ind[n]+1), n+1, cost[n-2])

    return point, threshold, slopes, cost, demands

def solve(point, threshold, slope, cost, demands):
    n = len(point)-2
    solution = np.zeros(n)

    for i in range(1,n+1):
        if i == 1:
            for j in range(slope.size):
                if slope.access(j)<=0 and slope.access(j+1)>=0:
                    solution[n-i] = point[j] 
                    break
        else:
            if solution[n-i+1]+ demands[n-i+1]<= threshold[n-i][0]:
                solution[n-i] = threshold[n-i][0]
            elif threshold[n-i][0]< solution[n-i+1]+ demands[n-i+1] and solution[n-i+1]+ demands[n-i+1] <  threshold[n-i][1]:
                solution[n-i] = solution[n-i+1]+ demands[n-i+1]
            else:
                solution[n-i] = threshold[n-i][1]
    #print(solution)
    final = 0
    for i in range(n):
        final = final + cost[i]*abs(solution[i])
        if i == 0:
            final = final + abs(solution[0] + demands[0]) 
        if i == n-1:
            final = final + abs(solution[n-1] - demands[n]) 
        else:
            final = final + abs(solution[i] -solution[i+1]-demands[i+1])
    return final

result = 0
for i in range(len(S_calibrated)):
    point, threshold, slope, cost , demands= DP(S_calibrated[i])
    result += solve(point, threshold, slope, cost, demands)
print(result/len(S_calibrated))


0.011329418691121366


In [639]:
def smCE_LP(S):
    (x_list, y_list) = S
    n = len(x_list)
    indices = np.argsort(x_list)
    x_list = x_list[indices]
    y_list = y_list[indices]
    A = np.diag([1]*len(x_list))
    A = (A -np.roll(A, 1, axis = 1))[0:len(x_list)-1]
    A = np.concatenate([A, -A])
    b = (x_list- np.roll(x_list,1,axis=0))[1:len(x_list)]
    b = np.concatenate([b,b])
    c = (y_list - x_list)
    return c, A, b



In [640]:
result = 0
for i in range(len(S_calibrated)):
    c, A, b = smCE_LP(S_calibrated[i])
    n = len(S_calibrated[i][0])
    x = cp.Variable(n)
    objective = cp.Minimize(np.array([c/n]) @ x)
    constraints = [-1 <= x, x <= 1, A@x <= b]
    prob = cp.Problem(objective, constraints)
    result += -prob.solve()
print(result/len(S_calibrated))


0.011329417598749591


In [641]:
def smCE_flow(S):
    (x_list, y_list) = S
    n = len(x_list)
    indices = np.argsort(x_list)
    x_list = x_list[indices]
    y_list = y_list[indices] 
    row = np.array(range(0,n-1))
    column = np.array(range(1,n)) 
    row = np.concatenate((row, np.array(range(1,n))))
    column = np.concatenate((column, np.array(range(0,n-1))))

    row = np.concatenate((row, np.array(range(0,n))))
    column = np.concatenate((column, np.array([n]*n)))
    row = np.concatenate((row, np.array([n]*n)))
    column = np.concatenate((column, np.array(range(0,n))))

    data = np.array([1]*len(row))
    
    cost = (x_list- np.roll(x_list,1,axis=0))[1:len(x_list)]
    cost = np.concatenate((cost,cost))
    cost = np.concatenate((cost, np.array([1]*2*n)))

    data = np.array([1]*len(row))
    graph = coo_matrix((data, (row, column)), shape=(n+1, n+1))
    capacities = coo_matrix((np.array([2]*len(row)), (row, column)), shape=(n+1, n+1)) 
    cost = coo_matrix((cost, (row, column)), shape=(n+1, n+1))

    c = np.array((y_list - x_list)/n)
    demands = np.concatenate((-c, np.array([np.sum(c)])))
    return graph, capacities, cost, demands

result_1 = 0
for i in range(len(S_calibrated)):
    graph, capacities, cost, demands = smCE_flow(S_calibrated[i])
    # print(graph.toarray())
    # print(capacities.toarray())
    # print(cost.toarray())
    # print(demands)
    obj, flow = min_cost_flow_scipy(graph, capacities, cost, demands, verbose=False)
    result_1+=obj
print(result_1/len(S_calibrated))


0.011329418372186423
